In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp2 = Experiment(workspace=ws, name="automl")

print('Workspace name: ' + ws.name,
 'Resource Group' + ws.resource_group, 
 'Location' + ws.location, 
 'Subscription ID' + ws.subscription_id, sep='\n')
run = exp2.start_logging()

Workspace name: quick-starts-ws-138517
Resource Groupaml-quickstarts-138517
Locationsouthcentralus
Subscription IDa24a24d5-8d87-4c8a-99b6-91ed2d2df51f


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster2="automl"
try:
  compute_target=ComputeTarget(workspace=ws,name=cluster2)
  print("Compute target exists")
except ComputeTargetException:
  print('Create a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
  compute_target=ComputeTarget.create(ws,cluster2,compute_config)
  

### YOUR CODE HERE ###

Compute target exists


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.core import Dataset
found = False
key="data"
# The dataset is from a Kaggle Competition: https://www.kaggle.com/blastchar/telco-customer-churn
#It has been uploaded as a CSV file to a github account

### YOUR CODE HERE ###
if key in ws.datasets.keys():
    found=True
    dataset=ws.datasets[key]

if not found:
    dataset=Dataset.Tabular.from_delimited_files(path="https://raw.githubusercontent.com/arfa-t/ML-Nanodegree/main/Capstone%20Project/churn.csv")
    #register it
    dataset=dataset.register(workspace=ws,
                            name=key)



In [4]:
from azureml.core.dataset import Dataset
from train import clean_data
import pandas as pd
x,y=clean_data(dataset)


In [5]:
#divide the cleaned data using ttain test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,shuffle=True)


In [6]:
x_test.head(1).columns

Index(['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year'],
      dtype='object')

In [7]:
pd.set_option('display.max_columns', None)


In [8]:
df = pd.concat([x, y], axis = 1)
df.head(5)

,SeniorCitizen,Partner,Dependents,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,Churn
0,0,True,False,False,True,29.85,29.85,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,False
1,0,False,False,True,False,56.95,1889.50,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,False
2,0,False,False,True,True,53.85,108.15,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,True
3,0,False,False,False,False,42.30,1840.75,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,False
4,0,False,False,True,True,70.70,151.65,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,True


In [9]:
import os
if not os.path.isdir('datadir'): #make a directory for data 
    os.mkdir('datadir')

# Convert the  data to a csv file to  upload it to the datastore
df.to_csv("datadir/data.csv", index=False)

dstore = ws.get_default_datastore()
dstore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-f2178614-b273-4035-aaa6-1c204af89d24",
  "account_name": "mlstrg138517",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [10]:
dstore.upload(src_dir='./datadir', target_path='automl-dataset', overwrite=True, show_progress=True)
 


Uploading an estimated of 1 files
Uploading ./datadir/data.csv
Uploaded ./datadir/data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_29e56b05ce7c48b68afceab226df6ffb

In [11]:
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
from azureml.core.dataset import Dataset

# Upload the training data as a tabular dataset so it can be used by automl compute
data = Dataset.Tabular.from_delimited_files(path=dstore.path('automl-dataset/data.csv'))
label = "Churn"

Since 5 folds cross validations are being performed, so for each training we use 4/5 th of data and
1/5 th of data is used in each validation  with a different holdout fold each time.

In [12]:
from azureml.train.automl import AutoMLConfig
import logging

automl_settings = {

       "verbosity": logging.INFO,
   }
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data,
    label_column_name='Churn',
    n_cross_validations=5,
    compute_target=compute_target,
    iterations=100,
    max_concurrent_iterations=8)

aml_run=exp2.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on automl with default configuration
Running on remote compute: automl
Parent Run ID: AutoML_12b401bf-484f-486b-81fa-3c7ad6c5feae

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://ak

In [13]:
aml_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_12b401bf-484f-486b-81fa-3c7ad6c5feae',
 'target': 'automl',
 'status': 'Completed',
 'startTimeUtc': '2021-02-11T13:29:22.505924Z',
 'endTimeUtc': '2021-02-11T14:10:53.756585Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-138517","workspace_name":"quick-starts-ws-138517","region":"southcentralus","compute_target":"automl","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto","_i

In [15]:
from azureml.widgets import RunDetails
RunDetails(aml_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## GET BEST MODEL

In [18]:
best_run,model=aml_run.get_output() #get best run and model


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [19]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_12b401bf-484f-486b-81fa-3c7ad6c5feae_98,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                multi_class='multinomial',
                                                                                                n_jobs=1,
                                                                                                penalty='l2',
                                                     

In [20]:
metrics=aml_run.get_metrics()
print(metrics)

{'experiment_status': ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection'], 'experiment_status_description': ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.'], 'average_precision_score_micro': 0.892206671929781, 'norm_macro_recall': 0.44085710962824576, 'recall_score_micro': 0.8091598500082396, 'precision_score_weighted': 0.8012895573950463, 'average_precision_score_weighted': 0.8661821575558031, 'f1_score_weighted': 0.800865859809052, 'weighted_accuracy': 0.8657353445996596, 'average_precision_score_macro': 0.8020373748432122, 'recall_score_macro': 0.7204285548141229, 'f1_score_micro': 0.8091598500082396, 'precision_score_micro': 0.8091598500082396, 'AUC_weighted': 0.848757

In [21]:
#Display the metrics in a more understandable manner
for i in metrics:
    print(i, metrics[i], '\n')


experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection'] 

experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.'] 

average_precision_score_micro 0.892206671929781 

norm_macro_recall 0.44085710962824576 

recall_score_micro 0.8091598500082396 

precision_score_weighted 0.8012895573950463 

average_precision_score_weighted 0.8661821575558031 

f1_score_weighted 0.800865859809052 

weighted_accuracy 0.8657353445996596 

average_precision_score_macro 0.8020373748432122 

recall_score_macro 0.7204285548141229 

f1_score_micro 0.8091598500082396 

precision_score_micro 0.8091598500082396 

AUC_weighted 0.8487575986628118 

log_loss 0.416623

In [22]:
model=run.register_model(model_name='bestaml.pkl', model_path='./outputs/') #register it

In [23]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-138517', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-138517'), name=bestaml.pkl, id=bestaml.pkl:1, version=1, tags={}, properties={})

In [ ]:
compute_target.delete()